# Overview
Student Success is often measured by 150% graduation rate (citation). The racial disparities in these numbers show the postsecondary educational gaps. Colleges will want to increase their black student success rate. 

# Business Understanding
Colleges who puport to help low income and minority students are often in a tight spot. Because they are putting extra dollars to help those students without, their margins are thinner. Consequently their graduation rates, or student sucess rates are lower than private schools who benefit from selecting better students and supporting them with wealthier students tuition or endowments. 

In pilling through IPEDS instiutional financial data, this project offers financial advice to public universities who are struggling to graduate black students, or would like to do better. 

# Data Understanding
Limitations. The graduation rate for IPEDS only calculates for first time, full-time students. Excludes transfers, winter enrollment, and part-timers. Instituion focused, not the individual students. hence the before, only students who start and finish at the university are included. 
No income data on students...can't focus on low income students. 

In [486]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pyodbc
import matplotlib.pyplot as plt

from functools import reduce
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

### Retrieval

IPEDS is from the Integrated Postsecondary Educatnoi Data System, accessed through yearly Microsoft Access databases and pyodbc. Six years (2014-2019) of financial data from 1700 public and private universities were pulled. After the queries were saved to csv, they are pulled to dataframe here. One advantage of IPEDS is that all tables have the same key: 'UNITID,' except for the provisional tables in 2019-2020. 

In [487]:
# setting up the connection
conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            r'DBQ=data/IPEDS201920.accdb;')
conn= pyodbc.connect(conn_str)
#example: pulling the target variable
df = pd.read_sql('select UNITID, GBA6RTT from DRVGR2019', conn)
# #save to csv, example "target_2019"
df.to_csv("data/target_2019.csv")

In [488]:
#pull financials
df_private = pd.read_csv("data/private_financials.csv").iloc[:, 1:]
df_public = pd.read_csv("data/public_financials.csv").iloc[:, 1:]
df_ids = pd.read_csv("data/university_ids.csv").iloc[:,0:2]
df_ids.rename(columns={'IPEDS\nUnit ID': 'UNITID', 'Organization or School Name':'School'}, inplace=True)

# Data Cleaning

Many of the financial columns are just the same data in relation to full time employers (FTE)(ending in FT) or in percentage of revenue/expenses (ending in PC). These columns are typically collinear. This project chose those FT columns, because of the efficiency of the metric in comparing universities per full time employees. 

Getting rid of columns such as, but not limited to, F1TUFEPC, F1STAPPC, F1LCAPPC, F1GVGCPC, etc.

### Feature Engineering
combining  income (from something other than tuition), and instruction, research, and institutional support and endowment expenses

In [489]:
#public income other than tuition -- F1GVGCFT, F1PGGCFT, F1INVRFT, F1OTRVFT
df_public['Rev_not_tuition'] =df_public['F1GVGCFT'] * df_public['F1PGGCFT'] *df_public['F1INVRFT'] *df_public['F1OTRVFT']
df_public.drop(columns=['F1GVGCFT', 'F1PGGCFT', 'F1INVRFT', 'F1OTRVFT'], axis=1)

#public instructional expenses -- F1INSTFT, F1RSRCFT,F1INSUFT, F1ENDMFT
df_public['Instruction_and_research'] =df_public['F1INSTFT'] *df_public['F1RSRCFT'] *df_public['F1INSUFT'] *df_public['F1ENDMFT']
df_public.drop(columns=['F1INSTFT', 'F1RSRCFT','F1INSUFT', 'F1ENDMFT'], axis=1)

#private research and endowment expenses -- F2RSRCFT,F2ENDMFT
df_private['Research_and_endowment']= df_private['F2RSRCFT'] *df_private['F2ENDMFT']
df_private.drop(columns=['F2RSRCFT','F2ENDMFT'], axis=1)

#private student support -- F2ACSPFT, F2STSVFT
df_private['Academic_and_student_support']= df_private['F2ACSPFT'] *df_private['F2STSVFT']
df_private.drop(columns=['F2ACSPFT', 'F2STSVFT'], axis=1)

,UNITID,F2A01,F2A19,F2A20,F2A02,F2A03,F2A03A,F2A04,F2A05,F2A05A,...,F2E115,F2E116,F2E117,F2E123,F2E124,F2E125,F2E126,F2E127,Research_and_endowment,Academic_and_student_support
0,100690,9384788.0,2634979.0,0.0,14438530.0,2487355.0,1296697.0,11242127.0,709048.0,174805.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100937,50743827.0,89400865.0,0.0,159285516.0,47756472.0,34000110.0,53054909.0,58474135.0,7669338.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101189,20868071.0,53530199.0,0.0,89941892.0,42026935.0,33512123.0,29345184.0,18569773.0,13162812.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101365,0.0,3242987.0,0.0,9435073.0,3717479.0,0.0,5717594.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101435,50818280.0,29298003.0,0.0,86309501.0,26153792.0,22307594.0,9068001.0,51087708.0,47866409.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9344,459842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9345,459851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9346,460349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,485077.0
9347,475228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


### Labels

In [490]:
#University Labels
private_ids = [df_ids, df_private]
public_ids = [df_ids, df_public]
df_private= reduce(lambda x, y: pd.merge(x, y, on = 'UNITID'), private_ids)
df_public= reduce(lambda x, y: pd.merge(x, y, on = 'UNITID'), public_ids)
df_private.set_index(['School'], inplace=True)
df_public.set_index(['School'], inplace=True)


if the target was not present, the observation was droped. 
also, if the row or column contained no information, it was droped. 
Once the index was set to year and the ID, all nans were droped. 

#### Continuous Target

In [491]:
#if you don't have the target, why are you even here?
df_private = df_private[~df_private['GBA6RTBK'].isna()]
#and again for public
df_public = df_public[~df_public['GBA6RTBK'].isna()]

In [492]:
#private
y_private_continuous = df_private['GBA6RTBK']
X_private_continuous = df_private.drop(columns = ['GBA6RTBK'], axis = 1)
X_train_private_continuous, X_test_private_continuous, y_train_private_continuous, y_test_private_continuous = train_test_split(X_private_continuous, y_private_continuous, random_state = 42)
#and for public
y_public_continuous = df_public['GBA6RTBK']
X_public_continuous = df_public.drop(columns = ['GBA6RTBK'], axis = 1)
X_train_public_continuous, X_test_public_continuous, y_train_public_continuous, y_test_public_continuous = train_test_split(X_public_continuous, y_public_continuous, random_state = 42)


Target-- ternanry and continuous
classification and regressoion

In [493]:
#ternary bining
bin_labels =['low', 'medium', 'high']
df_private['gr_rank'] = pd.qcut(df_private['GBA6RTBK'], q=3, labels=bin_labels)
#for public
df_public['gr_rank'] = pd.qcut(df_public['GBA6RTBK'], q=3, labels=bin_labels)

In [494]:
#private
y_private = df_private['gr_rank']
X_private = df_private.drop(columns = ['GBA6RTBK', 'gr_rank'], axis = 1)
X_train_private, X_test_private, y_train_private, y_test_private = train_test_split(X_private, y_private, random_state = 42)
#and for public
y_public = df_public['gr_rank']
X_public = df_public.drop(columns = ['GBA6RTBK', 'gr_rank'], axis = 1)
X_train_public, X_test_public, y_train_public, y_test_public = train_test_split(X_public, y_public, random_state = 42)


# Methods

### Ternary

In [498]:
#Set up pipeline for scaling continuous variables
pipeline_private= Pipeline(steps=[
    ('si', SimpleImputer()),
    ('ss', StandardScaler())
])
#Pipeline for running the model
dummy_private = Pipeline(steps=[
    ('pip', pipeline_private),
    ('dummy', DummyClassifier(random_state = 42))
])
#Fitting and checking the score
dummy_private.fit(X_train_private, y_train_private)
dummy_private.score(X_train_private, y_train_private)


0.33511111111111114

In [499]:
model_one_private = Pipeline(steps=[
    ('pip', pipeline_private),
    ('simple_dt', DecisionTreeClassifier(max_depth = 5, random_state = 42))
])
#Fit model on all the data
model_one_private.fit(X_train_private, y_train_private)

Pipeline(steps=[('pip',
                 Pipeline(steps=[('si', SimpleImputer()),
                                 ('ss', StandardScaler())])),
                ('simple_dt',
                 DecisionTreeClassifier(max_depth=5, random_state=42))])

In [500]:
#for public
pipeline_public = Pipeline(steps=[
    ('si', SimpleImputer()),
    ('ss', StandardScaler())
])
#Public
dummy_public = Pipeline(steps=[
    ('pip', pipeline_public),
    ('dummy', DummyClassifier(random_state = 42))
])
#Fitting and checking the score
dummy_public.fit(X_train_public, y_train_public)
dummy_public.score(X_train_public, y_train_public)

0.3442113442113442

In [501]:
model_one_public = Pipeline(steps=[
    ('pip', pipeline_public),
    ('simple_dt', DecisionTreeClassifier(max_depth = 5, random_state = 42))
])

#Fit model on all the data
model_one_public.fit(X_train_public, y_train_public)

Pipeline(steps=[('pip',
                 Pipeline(steps=[('si', SimpleImputer()),
                                 ('ss', StandardScaler())])),
                ('simple_dt',
                 DecisionTreeClassifier(max_depth=5, random_state=42))])

In [502]:
#Grab predictions and print precision
y_pred_private = model_one_private.predict(X_train_private)
print("Training Score:" + str(accuracy_score(y_train_private, y_pred_private)))
#Run a cross validation to test for overfitting
scores_private = np.mean(cross_val_score(model_one_private, X_train_private, y_train_private, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores_private))

Training Score:0.6551111111111111
Validation Score:0.6057777777777777


In [503]:
#Grab predictions and print precision
y_pred_public = model_one_public.predict(X_train_public)
print("Training Score:" + str(accuracy_score(y_train_public, y_pred_public)))
#Run a cross validation to test for overfitting
scores_public = np.mean(cross_val_score(model_one_public, X_train_public, y_train_public, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores_public))

Training Score:0.6507381507381508
Validation Score:0.5842869555362473


In [504]:
def sort_tuple(tup):
    tup.sort(reverse=True, key=lambda x:x[1])
    return tup

In [505]:
#Feature Importance
important_private = []
names_private = []
for name, importance in zip(X_train_private.columns, model_one_private['simple_dt'].feature_importances_):
    if importance > 0:
        important_private.append((name, importance))
        names_private.append(name)

print(sort_tuple(important_private))

[('F2A06', 0.4653450055914487), ('F2A18', 0.12289419261027125), ('F2A05A', 0.09496353262754326), ('F2INSTFT', 0.09160673141915081), ('F2C01', 0.03788333731547315), ('F2C08', 0.037537985051071095), ('F2E133', 0.02576173929007994), ('F2CORREV', 0.016276470253849425), ('F2E041', 0.011763869624225716), ('F2H01', 0.011193182155989004), ('F2TUFEFT', 0.01073886132953051), ('F2D083B', 0.010360784457951255), ('F2E071', 0.009781959535522562), ('F2A05', 0.009542115680950609), ('F2STSVFT', 0.008529016164702157), ('F2E131', 0.006400055635056215), ('F2E012', 0.006329523389921599), ('F2A03', 0.006251877026941996), ('F2ACSPPC', 0.005721576405568242), ('F2E056', 0.005368992421025483), ('F2B07', 0.0019924113164072887), ('F2E107', 0.001983729785616787), ('F2D05', 0.0017730509117029683)]


In [506]:
#For Public
important_public = []
names_public = []
for name, importance in zip(X_train_public.columns, model_one_public['simple_dt'].feature_importances_):
    if importance > 0:
        important_public.append((name, importance))
        names_public.append(name)

print(sort_tuple(important_public))

[('F1B05', 0.3449286397829274), ('F1C072', 0.19057652606475456), ('F1E01', 0.08546957972369444), ('F1A10', 0.07128870676181848), ('F1INSTFT', 0.039343018410130264), ('F1E09', 0.029257810835965), ('F1C061', 0.02898491424146599), ('F1TUFEFT', 0.02646382128737476), ('F1A284', 0.024537955144165104), ('F1B11', 0.022995867540547683), ('F1B13', 0.02085825019856816), ('F1C022', 0.019212625912713905), ('F1E02', 0.019024781862010388), ('F1E03', 0.01898452452669583), ('F1STAPFT', 0.013328816136141767), ('F1C112', 0.011688313588558636), ('F1M02', 0.008532699234351772), ('F1A214', 0.005235584652843107), ('UNITID', 0.004681554530849134), ('F1OTRVFT', 0.004356798576361169), ('F1ACSPPC', 0.003440527057582614), ('F1C013', 0.003425486946390443), ('F1A344', 0.0033831969840893024)]


### Update feature selection

In [507]:
names_private

['F2A03',
 'F2A05',
 'F2A05A',
 'F2A06',
 'F2A18',
 'F2B07',
 'F2C01',
 'F2C08',
 'F2D05',
 'F2D083B',
 'F2E012',
 'F2E041',
 'F2E071',
 'F2E131',
 'F2E133',
 'F2H01',
 'F2CORREV',
 'F2TUFEFT',
 'F2ACSPPC',
 'F2INSTFT',
 'F2STSVFT',
 'F2E056',
 'F2E107']

In [508]:
X_train_private = X_train_private[['F2A03',
 'F2A05',
 'F2A05A',
 'F2A06',
 'F2A18',
 'F2B07',
 'F2C01',
 'F2C08',
 'F2D05',
 'F2D083B',
 'F2E012',
 'F2E041',
 'F2E071',
 'F2E131',
 'F2E133',
 'F2H01',
 'F2CORREV',
 'F2TUFEFT',
 'F2ACSPPC',
 'F2INSTFT',
 'F2STSVFT',
 'F2E056',
 'F2E107']]

In [509]:
names_public

['UNITID',
 'F1A10',
 'F1A214',
 'F1A284',
 'F1A344',
 'F1B05',
 'F1B11',
 'F1B13',
 'F1C022',
 'F1C061',
 'F1C072',
 'F1C112',
 'F1M02',
 'F1E01',
 'F1E02',
 'F1E03',
 'F1E09',
 'F1TUFEFT',
 'F1STAPFT',
 'F1OTRVFT',
 'F1ACSPPC',
 'F1INSTFT',
 'F1C013']

In [510]:
X_train_public = X_train_public[[ 'F1A10',
 'F1A214',
 'F1A284',
 'F1A344',
 'F1B05',
 'F1B11',
 'F1B13',
 'F1C022',
 'F1C061',
 'F1C072',
 'F1C112',
 'F1M02',
 'F1E01',
 'F1E02',
 'F1E03',
 'F1E09',
 'F1TUFEFT',
 'F1STAPFT',
 'F1OTRVFT',
 'F1ACSPPC',
 'F1INSTFT',
 'F1C013']]

## Logistic Regression

In [511]:
logreg_private = Pipeline(steps=[
    ('cont', pipeline_private),
    ('logr', LogisticRegression(multi_class="multinomial"))
])
#Fit model on all the data
logreg_private.fit(X_train_private, y_train_private)
#Grab predictions and print precision
y_pred_private = logreg_private.predict(X_train_private)
print("Training Score:" + str(logreg_private.score(X_train_private,y_train_private)))
scores = np.mean(cross_val_score(logreg_private, X_train_private, y_train_private, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores))

Training Score:0.6171111111111112
Validation Score:0.6119999999999999


In [512]:
print(logreg_private["logr"].coef_)

[[-0.1765165   0.54070401  1.42950564  0.33758491  1.51750933  0.33758491
  -0.76216375  0.96171881 -0.35051187  0.42768805 -1.30284806 -0.0172185
   0.98733112  0.41343881  0.13790308  1.62204778 -0.11611698  0.26568791
   0.0768435   0.50037792  0.00262768 -0.00970094 -0.28035849]
 [ 0.03804075 -1.25416526 -2.16529802  1.2235692  -1.82802419  1.2235692
   0.50211092 -1.35816759  0.19019193 -0.35371857  0.51584012 -0.35810975
  -1.12865214 -0.85418755 -0.02641991 -0.37462479  0.34725321 -0.22702243
  -0.04366786 -0.42332757  0.02892833  0.15092552  0.26357733]
 [ 0.13847574  0.71346124  0.73579237 -1.56115411  0.31051486 -1.56115411
   0.26005282  0.39644878  0.16031994 -0.07396948  0.78700794  0.37532824
   0.14132101  0.44074874 -0.11148317 -1.24742299 -0.23113624 -0.03866549
  -0.03317564 -0.07705034 -0.03155602 -0.14122457  0.01678116]]


In [513]:
logreg_public = Pipeline(steps=[
    ('cont', pipeline_public),
    ('logr', LogisticRegression(multi_class="multinomial"))
])
#Fit model on all the data
logreg_public.fit(X_train_public, y_train_public)
#Grab predictions and print precision
y_pred_public = logreg_public.predict(X_train_public)
print("Training Score:" + str(logreg_public.score(X_train_public,y_train_public)))
scores = np.mean(cross_val_score(logreg_public,X_train_public, y_train_public, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores))

Training Score:0.6064491064491064
Validation Score:0.5967239620717011


In [514]:
print(logreg_public["logr"].coef_)

[[ 4.13542086e-01 -1.64848302e-01 -9.26367929e-01 -3.64199840e-02
   1.52745333e+00  1.20158098e-01 -1.57263166e-01  5.21209129e-02
   2.78654558e-01  5.58684811e-01 -2.26641023e-01  1.17447053e-01
  -2.45413304e-01  1.32604823e-03  5.77635633e-01  3.26524223e-01
   1.68245617e-01  1.77235487e-01  1.81271421e-01  1.05870244e-01
   8.81520139e-02 -1.48490622e-01]
 [-4.47272838e-01  2.85256083e-01  1.04360251e+00 -3.53823662e-03
  -2.26338875e+00 -8.47321385e-02 -5.47604966e-02 -5.67484464e-02
  -2.16090842e-01 -3.73006830e-01  3.26174106e-01 -2.80759311e-01
   2.42347357e-01  8.03873912e-02 -7.36827107e-01 -2.47806598e-01
  -2.51937889e-01 -1.53032566e-01 -2.72019928e-01 -2.98712116e-02
  -1.33743850e-01  7.58058010e-02]
 [ 3.37307519e-02 -1.20407781e-01 -1.17234583e-01  3.99582206e-02
   7.35935422e-01 -3.54259599e-02  2.12023662e-01  4.62753355e-03
  -6.25637153e-02 -1.85677981e-01 -9.95330833e-02  1.63312258e-01
   3.06594692e-03 -8.17134394e-02  1.59191475e-01 -7.87176249e-02
   8.3

## Regression

In [515]:
#Set up pipeline for scaling continuous variables
continuous_pipeline_private= Pipeline(steps=[
    ('si', SimpleImputer()),
    ('ss', StandardScaler())
])
lr_private = Pipeline(steps=[
    ('cont', continuous_pipeline_private),
    ('ols', LinearRegression())
])
#Fit model on all the data
lr_private.fit(X_train_private_continuous, y_train_private_continuous)
#Grab predictions and print precision
y_pred_private_continuous = lr_private.predict(X_train_private_continuous)
print("Training Score:" + str(lr_private.score(X_train_private_continuous, y_train_private_continuous)))
#Run a cross validation to test for overfitting
scores_private_continuous = np.mean(cross_val_score(lr_private, X_train_private_continuous, y_train_private_continuous, cv=5))
print("Validation Score:" + str(scores_private_continuous))

Training Score:0.41865974902826686
Validation Score:-4.1059669609815754e+26


In [516]:
#for public
continuous_pipeline_public = Pipeline(steps=[
    ('si', SimpleImputer()),
    ('ss', StandardScaler())
])
lr_public = Pipeline(steps=[
    ('cont', continuous_pipeline_public),
    ('ols', LinearRegression())
])
#Fit model on all the data
lr_public.fit(X_train_public_continuous, y_train_public_continuous)
#Grab predictions and print precision
y_pred_private_continuous = lr_public.predict(X_train_public_continuous)
print("Training Score:" + str(lr_public.score(X_train_public_continuous, y_train_public_continuous)))
#Run a cross validation to test for overfitting
scores_public_continuous = np.mean(cross_val_score(lr_public, X_train_public_continuous, y_train_public_continuous, cv=5))
print("Validation Score:" + str(scores_public_continuous))

Training Score:0.49743286899784944
Validation Score:-6842506773336856.0


In [521]:
#Feature Importance
important_private = []
for name, coef in zip(X_train_private_continuous.columns, lr_private['ols'].coef_):
    if coef > 0:
        important_private.append((name, coef))
print(sort_tuple(important_private))

[('F2D162', 40363470549928.875), ('F2A06', 35942943648374.49), ('F2D07', 35739061716768.11), ('F2D172', 21724534483687.773), ('F2D173', 14895185779081.898), ('F2E114', 12448795702367.916), ('F2A05B', 10387772384558.506), ('F2D022', 10229574102656.969), ('F2D183', 9201579590100.605), ('F2D163', 6850868398863.648), ('F2D164', 6677756852333.558), ('F2E091', 6098380546000.099), ('F2D083', 6085488053525.62), ('F2D08B', 5816084553419.051), ('F2D15', 5497146782054.066), ('F2A12', 5386692394558.335), ('F2D05', 5142420176619.747), ('F2D102', 5067302827806.227), ('F2A03', 5032860485421.64), ('F2D09', 4881978209902.959), ('F2D13', 4853935907191.285), ('F2D04', 4741391253186.146), ('F2D06', 4740778399283.527), ('F2E011', 4333064871103.756), ('F2C06', 4304519046949.309), ('F2E014', 4235454057345.305), ('F2A05A', 4079451370076.6523), ('F2D084A', 3195557012965.509), ('F2E021', 3022178499865.8975), ('F2D084', 2893644178780.6094), ('F2E024', 2644364822671.117), ('F2D03', 2579668949792.756), ('F2E064', 

In [522]:
#Feature Importance
important_public = []
for name, coef in zip(X_train_public_continuous.columns, lr_public['ols'].coef_):
    if coef > 0:
        important_public.append((name, coef))
print(sort_tuple(important_public))

[('F1A05', 8964397430443.81), ('F1C086', 2125143990970.2334), ('F1A09', 1879835942888.337), ('F1C197', 1856894066247.9224), ('F1E07', 1812561832022.8342), ('F1E10', 1361116148622.3179), ('F1C085', 1328348886439.0034), ('F1A18', 1267095783864.6028), ('F1A01', 1173137352259.0464), ('F1A11', 934971413145.3087), ('F1C191', 870797342498.3925), ('F1B23', 763103044490.8269), ('F1A10', 714598925507.0208), ('F1D06', 603045753801.1238), ('F1C082', 517866443133.72595), ('F1B19', 448720636497.30255), ('F1A234', 436014711776.7397), ('F1C122', 360227593087.8633), ('F1B20', 304512827356.0935), ('F1C012', 292000311437.3684), ('F1B21', 277132569651.4617), ('F1C083', 226000359372.84644), ('F1B04', 194798910530.41852), ('F1C124', 177253197246.1604), ('F1C125', 172971141110.74878), ('F1B22', 161326213135.8686), ('F1C114', 157973741492.63428), ('F1C022', 157096938979.10303), ('F1C014', 153349082044.46875), ('F1A324', 135534520903.70837), ('F1B06', 135148846753.92346), ('F1C084', 135062999938.89877), ('F1C0

In [ ]:
import statsmodels.api as sm
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            #if verbose:
                #print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            #if verbose:
                #print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    included.append('id')
    print('resulting features:')
    print(included)
    
    return included

In [529]:
stepwise_selection(X_train_private_continuous,y_train_private_continuous)


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [527]:
stepwise_selection(X_train_public_continuous,y_train_public_continuous)

MissingDataError: exog contains inf or nans

In [530]:
model_private = sm.OLS(y_train_private_continuous, X_train_private_continuous).fit()
model_private.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               GBA6RTBK   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     18.65
Date:                Wed, 01 Dec 2021   Prob (F-statistic):               0.00
Time:                        21:22:44   Log-Likelihood:                -19802.
No. Observations:                4500   AIC:                         3.998e+04
Df Residuals:                    4311   BIC:                         4.119e+04
Df Model:                         188                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -1.139e-05   5.52e-06     -2.065      0.039   -2.22e-05   -5.74e-07
x2          5.208e-09    5.1e-09      1.021      0.307    -4.8e-09    1.52e-08
x3             0.0032      0.002      1.679      0.093      -0.001       0.007
x4          9.901e-08   3.61e-08      2.746      0.006    2.83e-08     1.7e-07
x5            -0.0004      0.000     -2.011      0.044      -0.001   -1.12e-05
x6             0.0004      0.000      2.011      0.044    1.12e-05       0.001
x7          9.431e-09   7.82e-09      1.205      0.228   -5.91e-09    2.48e-08
x8            -0.0005      0.000     -2.010      0.045      -0.001   -1.31e-05
x9            -0.0004      0.000     -2.009      0.045      -0.001    -8.6e-06
x10           -0.0002   8.85e-05     -2.011      0.044      -0.000    -4.5e-06
x11           -0.0002   8.85e-05     -2.011      0.044      -0.000   -4.51e-06
x12           -0.0009      0.000     -2.011      0.044      -0.002   -2.25e-05
x13           -0.0005      0.000     -1.679      0.093      -0.001    9.06e-05
x14           -0.0005      0.000     -1.679      0.093      -0.001    9.07e-05
x15           -0.0005      0.000     -1.679      0.093      -0.001    9.06e-05
x16           -0.0005      0.000     -1.679      0.093      -0.001    9.07e-05
x17           -0.0005      0.000     -1.678      0.093      -0.001    9.07e-05
x18           -0.0027      0.002     -1.679      0.093      -0.006       0.000
x19            0.0032      0.002      1.679      0.093      -0.001       0.007
x20            0.0092      0.005      2.011      0.044       0.000       0.018
x21           -0.0092      0.005     -2.011      0.044      -0.018      -0.000
x22            0.0092      0.005      2.011      0.044       0.000       0.018
x23           -0.0064      0.003     -2.011      0.044      -0.013      -0.000
x24            0.0028      0.001      2.011      0.044    6.91e-05       0.005
x25            0.0028      0.001      2.011      0.044    6.91e-05       0.005
x26           -0.0009      0.000     -2.011      0.044      -0.002   -2.23e-05
x27        -1.287e-07   2.53e-07     -0.509      0.611   -6.24e-07    3.67e-07
x28        -1.723e-06   4.78e-07     -3.608      0.000   -2.66e-06   -7.87e-07
x29         6.725e-08   2.82e-07      0.239      0.811   -4.85e-07    6.19e-07
x30         1.316e-06   1.41e-06      0.931      0.352   -1.46e-06    4.09e-06
x31          6.44e-08   2.46e-07      0.262      0.794   -4.18e-07    5.47e-07
x32         7.002e-08    2.4e-07      0.292      0.771   -4.01e-07    5.41e-07
x33        -6.104e-08    2.4e-07     -0.254      0.799   -5.31e-07    4.09e-07
x34         1.562e-07   8.34e-08      1.874      0.061   -7.24e-09     3.2e-07
x35        -3.132e-07   1.89e-07     -1.656      0.098   -6.84e-07    5.76e-08
x36        -1.187e-08   5.16e-08     -0.230      0.818   -1.13e-07    8.93e-08
x3

In [531]:
model_public = sm.OLS(y_train_public_continuous, X_train_public_continuous).fit()
model_public.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               GBA6RTBK   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                     13.46
Date:                Wed, 01 Dec 2021   Prob (F-statistic):          3.15e-232
Time:                        21:22:55   Log-Likelihood:                -10624.
No. Observations:                2574   AIC:                         2.157e+04
Df Residuals:                    2414   BIC:                         2.250e+04
Df Model:                         159                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.606e-05   5.42e-06      2.965      0.003    5.44e-06    2.67e-05
x2         -1.108e-06   1.15e-06     -0.962      0.336   -3.37e-06    1.15e-06
x3         -8.431e-07   8.19e-07     -1.029      0.304   -2.45e-06    7.64e-07
x4         -8.409e-07    8.2e-07     -1.026      0.305   -2.45e-06    7.67e-07
x5         -2.736e-07   3.48e-07     -0.787      0.431   -9.55e-07    4.08e-07
x6          8.763e-07   1.16e-06      0.757      0.449   -1.39e-06    3.15e-06
x7         -2.407e-07   2.97e-07     -0.811      0.418   -8.23e-07    3.41e-07
x8         -1.018e-07    1.8e-07     -0.566      0.571   -4.54e-07    2.51e-07
x9         -4.348e-08    1.8e-07     -0.242      0.809   -3.96e-07    3.09e-07
x10        -4.389e-08   1.51e-07     -0.290      0.772    -3.4e-07    2.53e-07
x11         4.437e-07   2.75e-07      1.611      0.107   -9.65e-08    9.84e-07
x12         4.327e-07   2.76e-07      1.568      0.117   -1.08e-07    9.74e-07
x13        -5.119e-07   5.24e-07     -0.977      0.329   -1.54e-06    5.16e-07
x14         3.115e-07   3.48e-07      0.896      0.370    -3.7e-07    9.93e-07
x15         2.364e-07   3.02e-07      0.784      0.433   -3.55e-07    8.28e-07
x16        -8.914e-07   4.29e-07     -2.077      0.038   -1.73e-06   -4.99e-08
x17        -8.564e-07   4.29e-07     -1.995      0.046    -1.7e-06   -1.47e-08
x18        -9.138e-07   4.29e-07     -2.129      0.033   -1.76e-06   -7.23e-08
x19        -8.889e-07   4.29e-07     -2.071      0.039   -1.73e-06   -4.71e-08
x20        -3.701e-06   1.56e-06     -2.378      0.017   -6.75e-06   -6.49e-07
x21        -4.244e-07   3.74e-07     -1.135      0.257   -1.16e-06    3.09e-07
x22         -3.97e-07   3.74e-07     -1.062      0.288   -1.13e-06    3.36e-07
x23        -4.037e-07   3.74e-07     -1.080      0.280   -1.14e-06    3.29e-07
x24        -4.087e-07   3.74e-07     -1.093      0.275   -1.14e-06    3.25e-07
x25        -3.971e-07   3.73e-07     -1.063      0.288   -1.13e-06    3.35e-07
x26         4.133e-07   3.73e-07      1.107      0.268   -3.19e-07    1.15e-06
x27        -2.599e-08   8.59e-09     -3.026      0.003   -4.28e-08   -9.15e-09
x28        -7.117e-08   7.18e-08     -0.992      0.321   -2.12e-07    6.95e-08
x29          1.48e-08   1.58e-08      0.934      0.350   -1.63e-08    4.59e-08
x30         3.681e-06   1.19e-06      3.095      0.002    1.35e-06    6.01e-06
x31        -3.678e-06   1.19e-06     -3.094      0.002   -6.01e-06   -1.35e-06
x32         6.156e-06   2.28e-06      2.695      0.007    1.68e-06    1.06e-05
x33         9.836e-06   3.38e-06      2.914      0.004    3.22e-06    1.65e-05
x34         9.837e-06   3.38e-06      2.915      0.004    3.22e-06    1.65e-05
x35        -5.016e-06   1.62e-06     -3.092      0.002    -8.2e-06   -1.83e-06
x36          -4.9e-07   1.57e-07     -3.121      0.002   -7.98e-07   -1.82e-07
x3

### Assumptions
Accuracy 
Check assumption: homoscedasticity of residuals, normality of residualts, no multicollinearity

In [ ]:
#Grab probabilities and calculate log odds
pred = logreg_private.predict_proba(X_train_private)[:, 0]
log_odds = np.log(pred / (1 - pred))
#Plot log odds versus continuous variable to check for linearity
plt.scatter(x = X_train_private[''].values, y = log_odds)
plt.title("Logistic Regression Assumption Test")
plt.xlabel("")
plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
plt.ylabel("Log-odds")
plt.show();

KeyError: ''

In [ ]:
#Grab probabilities and calculate log odds
pred = logreg_public.predict_proba(X_train_public)[:, 0]
log_odds = np.log(pred / (1 - pred))
#Plot log odds versus continuous variable to check for linearity
plt.scatter(x = X_train_public[''].values, y = log_odds)
plt.title("Logistic Regression Assumption Test")
plt.xlabel("")
plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
plt.ylabel("Log-odds")
plt.show();

# Evaluation

### Private

In [ ]:
print("Training Accuracy:" + str(logreg_private.score(X_train_private, y_train_private)))
scores = np.mean(cross_val_score(logreg_private, X_train_private, y_train_private, cv=5))
print("Validation Accuracy:" + str(scores))

Training Accuracy:0.6515555555555556
Validation Accuracy:0.6113333333333333


In [ ]:
#Generate precision score for test set
test_pred = logreg_private.predict(X_test_private)
print("Test Accuracy:" + str(logreg_private.score(X_test_private, y_test_private)))

Test Accuracy:0.624


### Public

In [ ]:
print("Training Accuracy:" + str(logreg_public.score(X_train_public, y_train_public)))
scores = np.mean(cross_val_score(logreg_public, X_train_public, y_train_public, cv=5))
print("Validation Accuracy:" + str(scores))

Training Accuracy:0.6923076923076923
Validation Accuracy:0.6134501907748102


In [ ]:
#Generate precision score for test set
test_pred = logreg_public.predict(X_test_public)
print("Test Accuracy:" + str(logreg_public.score(X_test_public, y_test_public)))

Test Accuracy:0.6095571095571095


# Recommendations

# Next Steps
Look at loan default rates, only looking at GR exaggerates selection bias (E.g. Ivy's have the best GR). Schools with missions to help low-income students often have lower graduation rates. 